# Analysis

* Enter - edit
* M - switch to markdown
* Y - switch to code
* ctrl-enter - run cell
* A - add above
* B - add below

Shared functions for analysis

In [4]:
import re
import os

import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 20)

# constants
ALL_OUTPUT = "/home/dubs/dev/paap/data/output/calculate_modern_price_output.json"
PHOTOS_AND_PRINTS = "/home/dubs/dev/paap/data/output/photos_and_prints_only.json"
IMAGE_SIZES = "/home/dubs/dev/paap/data/output/image_dimensions_output.ndjson"
IMAGE_DIR = "/home/dubs/dev/paap/data/img/christies/s128/all"
FINAL_FILTER = "/home/dubs/dev/paap/data/output/final_is_usable.json"
OUT = "/home/dubs/dev/paap/data/output/christies.ndjson"

In [8]:
df_all = pd.read_json(ALL_OUTPUT, orient="records")

In [14]:
df_all.shape
# 259596 rows
df.columns
max(df.sale_year)

2019

In [8]:
image_sizes = pd.read_json(IMAGE_SIZES, orient="records", lines=True)

In [9]:
final_filter = pd.read_json(FINAL_FILTER, orient="records", lines=True)
final_filter.shape

(12500, 2)

In [10]:
 df_all.shape
 image_sizes.shape
 df = pd.merge(df_all, image_sizes, on="lot_image_id", how="left")
 df.shape

(259596, 26)

In [11]:
print(df_all.shape)
print(image_sizes.shape)
print(df.shape)

(259596, 24)
(360175, 3)
(259596, 26)


In [12]:
df_all

,id,sale_number,sale_url,sale_input_url,sale_year,sale_month,sale_date,sale_location,sale_location_int,sale_category,...,lot_estimate_low,lot_estimate_high,lot_description,lot_medium,lot_image_id,usd_equivalent,inflation_to_2020,lot_realized_price_usd,lot_estimate_low_usd,lot_estimate_high_usd
0,d9389924-38c7-4411-b042-345b2993e6e4,1756,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2006,10,2006-10-03,new_york,43,asian_art,...,5000,7000,"Bolognese School, 16th Century",,812944f7-7607-45f9-ac1b-966b14cfaad8,1.0,1.278350,6136.079286,6391.749257,8948.448959
1,5e53cd6f-cb93-45a9-bd65-191fe5b336d6,1756,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2006,10,2006-10-03,new_york,43,asian_art,...,5000,7000,After Correggio,,6a873c3b-1672-4996-b492-c04889326e2f,1.0,1.278350,5829.275322,6391.749257,8948.448959
2,8852dd94-6719-4522-8492-2da04e2fcd1b,1756,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2006,10,2006-10-03,new_york,43,asian_art,...,1000,1500,"AFTER RAPHAEL, 20TH CENTURY",,ec5fab69-3aeb-4473-bb1a-61566c969c05,1.0,1.278350,153.401982,1278.349851,1917.524777
3,0da60937-40f3-41c3-8681-865bc981a04e,1756,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2006,10,2006-10-03,new_york,43,asian_art,...,2000,3000,"After Andrea del Sarto, early 19th Century",,93366520-7112-47d9-bbff-f5e475b221a2,1.0,1.278350,2914.637661,2556.699703,3835.049554
4,eeba8512-4fd0-48d9-b212-5d6c84143d0e,1756,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2006,10,2006-10-03,new_york,43,asian_art,...,5000,7000,Circle of Jan Miel (Beveren-Waes 1599-1664 Turin),,f2e1e70c-a4da-4422-b0bb-a63466da5b16,1.0,1.278350,4602.059465,6391.749257,8948.448959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259591,1a576346-ceec-4b96-8fb9-6207264c0703,18084,https://onlineonly.christies.com/s/matisse-pap...,https://www.christies.com/results?sc_lang=en&m...,2019,3,2019-03-01,online,115,photographs_and_prints,...,7000,10000,Henri Matisse (1869-1954)<br>Pot au fleurs<br>...,None,c03b6bfe-a7b6-4445-845a-60d26fcef8fb,1.0,1.014827,19028.002337,7103.787539,10148.267913
259592,c2f91534-cecc-4350-b54a-e6d955a7535d,18084,https://onlineonly.christies.com/s/matisse-pap...,https://www.christies.com/results?sc_lang=en&m...,2019,3,2019-03-01,online,115,photographs_and_prints,...,8000,12000,HENRI MATISSE (1869-1954)<br>Nu renversé au br...,None,ef6cadc9-8d1c-4929-b1ed-6f077b2307ee,1.0,1.014827,30444.803739,8118.614330,12177.921496
259593,82e9e786-12df-48e4-b2dc-26663573452e,18084,https://onlineonly.christies.com/s/matisse-pap...,https://www.christies.com/results?sc_lang=en&m...,2019,3,2019-03-01,online,115,photographs_and_prints,...,8000,12000,"HENRI MATISSE (1869-1954)<br>Nu à genoux, bras...",None,05f36344-3821-43a1-b841-0ebf843cec48,1.0,1.014827,19028.002337,8118.614330,12177.921496
259594,2804252d-35ec-4b59-93b7-e1b6ec9be955,18084,https://onlineonly.christies.com/s/matisse-pap...,https://www.christies.com/results?sc_lang=en&m...,2019,3,2019-03-01,online,115,photographs_and_prints,...,5000,7000,HENRI MATISSE (1869-1954)<br>Nu assis au colli...,None,0939e479-83dd-49e6-bda0-22e002fd4200,1.0,1.014827,22833.602804,5074.133956,7103.787539


In [13]:
df_all["sale_category"].value_counts()

fine_art                        162081
photographs_and_prints           51173
asian_art                        42187
islamic_and_eastern_european      4155
Name: sale_category, dtype: int64

In [14]:
# df_all["lot_artist"].value_counts()[:100]
df_all["lot_description"].value_counts()[:100].values

array([4180, 2100, 1109, 1062, 1038,  870,  803,  634,  602,  593,  552,
        552,  524,  498,  470,  447,  432,  426,  402,  393,  367,  362,
        360,  353,  347,  336,  334,  333,  322,  315,  315,  315,  314,
        313,  308,  305,  304,  292,  291,  285,  277,  270,  267,  265,
        264,  257,  255,  247,  246,  243,  233,  232,  232,  228,  226,
        222,  219,  219,  215,  212,  211,  210,  207,  206,  205,  204,
        202,  201,  197,  196,  192,  191,  189,  189,  188,  188,  187,
        187,  182,  181,  180,  180,  177,  175,  170,  170,  169,  169,
        168,  165,  165,  163,  159,  156,  153,  152,  152,  152,  151,
        151])

One option would be to take the top x represented artists. You would then want
the model to be fairly adept at picking those artists. Would be interesting to
see which artworks were not selected by the model.

Another option would be to take a weighted random sample of the top n artists and
cluster to something mush smaller than that

Could also take all black and white prints and cluster on that. Might not have
enough data to do that well. - Only have 691 b&w photos, so no, don't have enough

In [15]:
# get all the images that we have
image_paths = [fp.split(".")[0] for fp in os.listdir(IMAGE_DIR) if os.path.isfile(os.path.join(IMAGE_DIR, fp))]
len(image_paths)

360175

In [16]:
# Filter to only where we have images
df = df[df.lot_image_id.isin(image_paths)]
df.shape

(256020, 26)

In [17]:
# Select the top 100 artists by number of instances in the dataset
MAKER_RE = re.compile(r"^(?P<maker>[\w\s\.,\-'']+)\s\([\s0-9bBb\-\.]+\)$")

def strip_date(x):
    match = re.search(MAKER_RE, x)
    if not match:
        return x
    return match.group("maker")

df["lot_description"] = df["lot_description"].apply(lambda x: x.lower().strip())
df = df[~df.lot_description.isin(["a", "a", "none" "", " ", "english school, 19th century", "after pablo picasso"])]
df["lot_description"] = df["lot_description"].apply(lambda x: x if not x.startswith("joan mir") 
else "joan miro")
df["lot_description"] = df["lot_description"].apply(strip_date)

# drop duplicates
duplicated_image_ids = df.duplicated(subset="lot_image_id", keep="last")
print("dropping {} duplicates".format(sum(duplicated_image_ids)))
df = df[~duplicated_image_ids]

duplicated_image_urls = df.duplicated(subset="lot_image_url", keep="last")
print("dropping {} duplicated image_urls".format(sum(duplicated_image_urls)))
df = df[~duplicated_image_urls]

top_100 = df["lot_description"].value_counts()[:100]
top_100

dropping 10889 duplicates
dropping 0 duplicated image_urls


pablo picasso                   4588
andy warhol                     2681
joan miro                       1822
marc chagall                    1452
rembrandt harmensz. van rijn     892
                                ... 
georg baselitz                   184
kees van dongen                  184
diane arbus                      184
cindy sherman                    183
chu teh-chun                     182
Name: lot_description, Length: 100, dtype: int64

In [18]:
# Black and White photos
# TODO Need to download files
df_bw = df[df["image_channels"] == 1]
df["image_channels"].value_counts()

3.0    239541
1.0       671
Name: image_channels, dtype: int64

In [19]:
top_50 = top_100[:50]
top_50

pablo picasso                   4588
andy warhol                     2681
joan miro                       1822
marc chagall                    1452
rembrandt harmensz. van rijn     892
                                ... 
max ernst                        290
christo                          289
auguste rodin                    287
wayne thiebaud                   279
pu ru                            270
Name: lot_description, Length: 50, dtype: int64

In [20]:
top_50_artists = list(top_50.index)
top_50_artists

['pablo picasso',
 'andy warhol',
 'joan miro',
 'marc chagall',
 'rembrandt harmensz. van rijn',
 'david hockney',
 'roy lichtenstein',
 'henri matisse',
 'albrecht dürer',
 'henry moore',
 'francis newton souza',
 'zhang daqian',
 'sam francis',
 'irving penn',
 'alexander calder',
 'robert rauschenberg',
 'frank stella',
 'damien hirst',
 'jasper johns',
 'lucio fontana',
 'jean dubuffet',
 'victor vasarely',
 'tom wesselmann',
 'pierre-auguste renoir',
 'zao wou-ki',
 'gerhard richter',
 'henri cartier-bresson',
 'sol lewitt',
 'henri de toulouse-lautrec',
 'bernard buffet',
 'keith haring',
 'raoul dufy',
 'qi baishi',
 'helmut newton',
 'jim dine',
 'ansel adams',
 'hiroshi sugimoto',
 'salvador dalí',
 'alighiero boetti',
 'robert motherwell',
 'edgar degas',
 'karel appel',
 'maurice de vlaminck',
 'laurence stephen lowry, r.a.',
 'takashi murakami',
 'max ernst',
 'christo',
 'auguste rodin',
 'wayne thiebaud',
 'pu ru']

In [21]:
# randomly sample n works from each artist
n_per_artist = 250
assert len(top_50_artists) == len(set(top_50_artists))
n_artists = len(top_50_artists)
n = int(n_per_artist * n_artists)

assert n_per_artist % int(n_per_artist) == 0
n_per_artist = int(n_per_artist)


artist_datasets = {}
for artist in top_50_artists:
    artist_datasets[artist] = df[df["lot_description"] == artist]

seed = 21
final_df = None
for _, v in artist_datasets.items():
    sampled = v.sample(n=n_per_artist, random_state=int(seed))
    if final_df is None:
        final_df = sampled
        continue

    final_df = pd.concat([final_df, sampled])

assert final_df.shape[0] == n
assert set(final_df["lot_description"].value_counts().values) == {n_per_artist}

In [22]:
pd.set_option("display.max_rows", 50)
final_df["lot_description"].value_counts()
final_df.columns

Index(['id', 'sale_number', 'sale_url', 'sale_input_url', 'sale_year',
       'sale_month', 'sale_date', 'sale_location', 'sale_location_int',
       'sale_category', 'sale_currency', 'lot_image_url', 'lot_artist',
       'lot_realized_price', 'lot_estimate_low', 'lot_estimate_high',
       'lot_description', 'lot_medium', 'lot_image_id', 'usd_equivalent',
       'inflation_to_2020', 'lot_realized_price_usd', 'lot_estimate_low_usd',
       'lot_estimate_high_usd', 'image_channels', 'image_shape'],
      dtype='object')

In [23]:
# Check for duplicated rows with id, lot_image_url, lot_image_id
sum(final_df.duplicated(subset="lot_image_id"))
# sum(final_df.duplicated(subset="lot_image_url"))
# sum(final_df.duplicated(subset="id"))

0

(12500, 2)

In [27]:
final_df.shape
final_filter.shape
final_out = pd.merge(final_df, final_filter, on="lot_image_id", how="inner")
final_out
final_out = final_out[final_out.lot_is_2d]
final_out.shape

(10505, 27)

In [30]:
pd.set_option("display.max_rows", 50)
final_out["lot_description"].value_counts().to_csv("~/Downloads/christies.csv")

In [28]:
# Write the output
final_out.to_json(OUT, orient="records", lines=True)

## Final Output Statistics

In [5]:
df = pd.read_json(OUT, lines=True)

,id,sale_number,sale_url,sale_input_url,sale_year,sale_month,sale_date,sale_location,sale_location_int,sale_category,...,lot_medium,lot_image_id,usd_equivalent,inflation_to_2020,lot_realized_price_usd,lot_estimate_low_usd,lot_estimate_high_usd,image_channels,image_shape,lot_is_2d
0,c70bd840-05bb-4b26-b636-8ca523567e5d,1719,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2006,10,2006-11-01,new_york,43,photographs_and_prints,...,,bd273067-e212-46a8-af93-3616110362e2,1.000000,1.278350,5.369069e+04,3.195875e+04,4.474224e+04,3,"(128, 128, 3)",True
1,42d4f926-04b3-475c-a333-681aac85c266,5300,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2007,10,2007-10-25,london,36,photographs_and_prints,...,,18b7460e-3368-4716-ad42-4752cef47fd7,2.040869,1.234689,6.299597e+03,5.039677e+03,7.559516e+03,3,"(128, 128, 3)",True
2,276e3133-1fc1-45a8-a010-2b64610588d1,10556,https://www.christies.com/Prints-and-Multiples...,https://www.christies.com/results?sc_lang=en&m...,2015,12,2015-12-10,london,36,photographs_and_prints,...,None,4ed60f4c-182f-4d0e-9f89-2cce166deb1f,1.506538,1.090671,3.902451e+03,3.286275e+03,4.929412e+03,3,"(128, 128, 3)",True
3,d18748d7-58b1-495c-afc1-fc81753b66ac,1102,https://www.christies.com/ImpressionistModern-...,https://www.christies.com/results?sc_lang=en&m...,2013,2,2013-02-06,london,36,fine_art,...,None,17c015bd-3dc6-48cf-af94-9e5b2488cc4d,1.583382,1.111149,2.634221e+06,1.583436e+06,2.111247e+06,3,"(128, 128, 3)",True
4,18bb76fd-50b0-42e7-8b5a-3d48f18e3c9c,2593,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2012,10,2012-10-31,new_york,43,photographs_and_prints,...,,7de05434-1648-4a13-979e-053206e84087,1.000000,1.115227,9.061221e+03,8.921817e+03,1.338273e+04,3,"(128, 128, 3)",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10500,9776555e-6ad6-4d1b-951c-c53b66dbc804,15887,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2018,9,2018-09-11,new_york,43,asian_art,...,,2c5636bb-9c0e-4fcd-80bc-38982b9a175f,1.000000,1.021914,5.748267e+04,2.656977e+04,3.985465e+04,3,"(128, 128, 3)",True
10501,d148655d-1339-4ff0-8719-482644f7a6cb,3313,https://www.christies.com/Fine-Chinese-Modern-...,https://www.christies.com/results?sc_lang=en&m...,2014,5,2014-05-27,hong_kong,35,asian_art,...,None,83147f8d-fbe3-474c-bd09-5e415ab99319,0.128982,1.084367,6.643554e+04,1.398643e+04,2.097964e+04,3,"(128, 128, 3)",True
10502,75d2df19-169b-48d8-95f2-d9d1b3b5e146,3313,https://www.christies.com/Fine-Chinese-Modern-...,https://www.christies.com/results?sc_lang=en&m...,2014,5,2014-05-27,hong_kong,35,asian_art,...,None,04e7183b-b419-4ce9-ae95-804c35844b32,0.128982,1.084367,1.398643e+04,1.118914e+04,1.398643e+04,3,"(128, 128, 3)",True
10503,eac44d19-c845-4777-b74c-dc80f07c2d7a,3422,https://www.christies.com/results?sc_lang=en&m...,https://www.christies.com/results?sc_lang=en&m...,2015,6,2015-06-02,hong_kong,35,asian_art,...,ink and colour on silk,4688d3cd-c89d-426a-b786-3018709d84a1,0.128949,1.081014,4.878835e+04,2.787905e+04,4.181858e+04,3,"(128, 128, 3)",True
